<a href="https://colab.research.google.com/github/SamTremblay18/Hockey-Shot-Classification/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries 

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from google.colab import drive 
drive.mount('/content/drive')
import pickle
from sklearn.preprocessing import LabelEncoder, MaxAbsScaler, StandardScaler
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.model_selection import train_test_split
from itertools import chain

Mounted at /content/drive


# Functions

Save and open files with pickle

In [3]:
def save_pickle(path_pickle,P):
  f = open(path_pickle, 'wb')
  pickle.dump(P, f)
  f.close()

  return f

def open_pickle(path_pickle):
  f = open(path_pickle, 'rb')
  P = pickle.load(f)
  f.close()

  return P  

Regroups all the Pre, SH and Post trials together to the label 'Other'

In [4]:
def regroup_other(P_markers,P_acc):
  for i in range(0,len(P_markers.iloc[:,0])):
    
    length_trial = P_markers.iloc[i,2] 
    start_trial = P_markers.iloc[i,0]  
   
    if P_markers.iloc[i,1] == 'Pre' or P_markers.iloc[i,1] == 'SH' or P_markers.iloc[i,1] == 'post':
      P_acc.loc[(start_trial):(start_trial)+((length_trial)-1),'Label'] = 'Other' # Swapping the label to Other

  return P_acc

Reframe all the shooting trials the same frame length

In [5]:
def reframe(P_markers,P_acc,f_length):
  for i in range(0,len(P_markers.iloc[:,0])):

    length_trial = P_markers.iloc[i,2]
    start_trial = P_markers.iloc[i,0]
    type_shot = P_markers.iloc[i,1]

    if P_markers.iloc[i,1] == 'BH' or P_markers.iloc[i,1] == 'Pass' or P_markers.iloc[i,1] == 'SS' or P_markers.iloc[i,1] == 'WS' or P_markers.iloc[i,1] == 'OT':
        diff = int(f_length-length_trial) # Identifying how many frames are missing to have the desired length
        P_acc.iloc[int(start_trial)-diff:int(start_trial)+int(length_trial),-1] = type_shot # Swapping the old labels from the difference 
                                                                                            # found above to the new label desired
  return P_acc

Standardize and label encoder function

In [6]:
def stand_labelencoder(P_acc):
  scaler = StandardScaler()
  le = LabelEncoder()
  scaled = scaler.fit_transform(P_acc.iloc[:,1:-1], y = P_acc['Label'])
  y = le.fit_transform(P_acc['Label'])
  classes = le.classes_
  y = y[:,None]
  x = np.concatenate((scaled, y), axis=1)

  return x, classes

Create tensors of specific shape

In [7]:
def create_tensor(P_acc,f_length):
  trials_length = len(P_acc)/f_length
  T = np.stack(np.split(P_acc, trials_length), axis=1)
  Tensors = tf.constant(T.astype('float32'))
  Tensors = tf.transpose(Tensors, perm = [1, 0, 2])

  return Tensors

Create 15 trials of 'Other' with specific length

In [8]:
# The majority of the data is in the category 'Other', so this function randomly creates
# 15 trials from data labeled as 'Other' with the specified length
def other_class(P_other,f_length):
  random.seed(4)
  num_list = random.sample(range(0,len(P_other),f_length),15) # this creates 15 numbers that are seperated with at least
                                                              # the specified length from all the data labeled as 'Other'
  Other_all = pd.DataFrame()
  for i in num_list:
    other = P_other.iloc[i:(i+f_length),:] # creates the trials with the specified length starting with number from the list
    Other_all = pd.concat([Other_all, other], ignore_index = True)

  return Other_all

Runs all the preprocessing steps for the reframe method

In [9]:
def Preprocess(P_markers, P_acc):
  P_data = regroup_other(P_markers,P_acc) # regroups the 'SH', 'Pre', 'post' trials to the label 'Other'
  P_data = reframe(P_markers,P_data,576) # reframe the dataset to 576 frames
  P_other = P_data[(P_data['Label'] == 'Other')] # creates a variable containing all the data labeled 'Other'
  P_other_all = other_class(P_other,576) # from the variable above, creates 15 trials of length 576
  P_data = P_data.drop(P_data[(P_data['Label'] == 'Do not use')].index) # removing all the data labeled as 'Do not use' (junk)
  P_data = P_data.drop(P_data[(P_data['Label'] == 'Other')].index) # removing all the data labeled as 'Other' from the original dataset
  P_data = pd.concat([P_data, P_other_all], ignore_index = True) # adding the 15 'Other' trials of length 576 to the original dataset 
  P_data, classes = stand_labelencoder(P_data)
  P_data = create_tensor(P_data,576) # transforming the dataset into tensors with a length of 576 frames
  
  return P_data, classes

Resample the shot trials to a set frequency

In [10]:
def resampling_data(markers_number, p_resample, p_acc, sampling_freq):

  for i in range(0,len(markers[markers_number].iloc[:-1,2])):

          trial_start = markers[markers_number].iloc[i,0]
          typeshot = markers[markers_number].iloc[i,1]
          trial_len = markers[markers_number].iloc[i,2]

          if typeshot == 'WS':
            # Resamples a trial by selecting all the data except the 'Frame' and 'label' column to a specified frequency
            data_resample[p_resample][f'WS_{i}'] = signal.resample(data_biomech_config[p_acc].iloc[int(trial_start):(int(trial_start)+int(trial_len)),1:-1],sampling_freq)
            # Adds the labels encoded manually at the last column position
            data_resample[p_resample][f'WS_{i}'] = np.append(data_resample[p_resample][f'WS_{i}'], np.full((sampling_freq,1), 6), axis=1)

          elif typeshot == 'BH':
            data_resample[p_resample][f'BH_{i}'] = signal.resample(data_biomech_config[p_acc].iloc[int(trial_start):(int(trial_start)+int(trial_len)),1:-1],sampling_freq)
            data_resample[p_resample][f'BH_{i}'] = np.append(data_resample[p_resample][f'BH_{i}'], np.full((sampling_freq,1), 0), axis=1)

          elif typeshot == 'OT':
            data_resample[p_resample][f'OT_{i}'] = signal.resample(data_biomech_config[p_acc].iloc[int(trial_start):(int(trial_start)+int(trial_len)),1:-1],sampling_freq)
            data_resample[p_resample][f'OT_{i}'] = np.append(data_resample[p_resample][f'OT_{i}'], np.full((sampling_freq,1), 1), axis=1)

          elif typeshot == 'SS':
            data_resample[p_resample][f'SS_{i}'] = signal.resample(data_biomech_config[p_acc].iloc[int(trial_start):(int(trial_start)+int(trial_len)),1:-1],sampling_freq)
            data_resample[p_resample][f'SS_{i}'] = np.append(data_resample[p_resample][f'SS_{i}'], np.full((sampling_freq,1), 5), axis=1)

          elif typeshot == 'Pass':
            data_resample[p_resample][f'Pass_{i}'] = signal.resample(data_biomech_config[p_acc].iloc[int(trial_start):(int(trial_start)+int(trial_len)),1:-1],sampling_freq)
            data_resample[p_resample][f'Pass_{i}'] = np.append(data_resample[p_resample][f'Pass_{i}'], np.full((sampling_freq,1), 3), axis=1)

          else:
            continue

#Method 1 - Reframe to 576 frames and All Sensors Configuration

In [ ]:
# Participants' and markers' dictionnary
data = open_pickle('/content/drive/MyDrive/Sam/File_saved/P_dct.pickle')
markers = open_pickle('/content/drive/MyDrive/Sam/File_saved/markers_dct.pickle')

In [ ]:
# Preprocess function for method 1 and storing into P_processed1 dictionary
P_processed1 = {}
for l,n in zip(markers, data):
  P_processed1[f'{n}_processed'], data_classes1 = Preprocess(markers[l],data[n])

In [ ]:
data_classes1

array(['BH', 'OT', 'Other', 'Pass', 'Rest', 'SS', 'WS'], dtype=object)

In [ ]:
# Subject-wise shuffling
keys = list(P_processed1.keys())
random.seed(10)
random.shuffle(keys)

SuffledP_processed1 = dict()
for key in keys:
    SuffledP_processed1.update({key: P_processed1[key]})

In [ ]:
# Splitting data and labels in train/test
train_values1 = list(SuffledP_processed1.values())[0:31]
x_train1 = tf.concat(train_values1, axis = 0)
X_train1 = x_train1[:,:,:-1].numpy()
y_train1 = x_train1[:,0,-1].numpy()

test_values1 = list(SuffledP_processed1.values())[31:39]
x_test1 = tf.concat(test_values1, axis = 0)
X_test1 = x_test1[:,:,:-1].numpy()
y_test1 = x_test1[:,0,-1].numpy()

print("X_train:",X_train1.shape,
      "X_test:",X_test1.shape,
      "y_train:",y_train1.shape,
      "y_test:",y_test1.shape)

X_train: (2638, 576, 51) X_test: (681, 576, 51) y_train: (2638,) y_test: (681,)


In [ ]:
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_test1.pickle',X_test1)
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_train1.pickle',X_train1)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_test1.pickle',y_test1)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_train1.pickle',y_train1)

<_io.BufferedWriter name='/content/drive/MyDrive/Sam/File_saved/y_train1.pickle'>

# Method 2 - Reframe to 576 frames and Biomechanics Sensor Configuration (BSC)
 

In [ ]:
data = open_pickle('/content/drive/MyDrive/Sam/File_saved/P_dct.pickle')
markers = open_pickle('/content/drive/MyDrive/Sam/File_saved/markers_dct.pickle')

In [ ]:
# Selecting the sensor configuration
data_BSC = {}
for i in data:
  data_BSC[f'{i}_bmech'] = data[i].loc[:,('Frame','Right Hand x','Right Hand y','Right Hand z','Left Hand x','Left Hand y','Left Hand z',
                                                     'Right Foot x','Right Foot y','Right Foot z','Left Foot x','Left Foot y','Left Foot z',
                                                     'Pelvis x','Pelvis y','Pelvis z','Label')]

In [ ]:
# Preprocess function for method 2 and storing into 'P_processed2' dictionary
P_processed2 = {}
for l,n in zip(markers, data_BSC):
  P_processed2[f'{n}_processed2'], data_classes2 = Preprocess(markers[l],data_BSC[n])

In [ ]:
data_classes2

array(['BH', 'OT', 'Other', 'Pass', 'Rest', 'SS', 'WS'], dtype=object)

In [ ]:
# Subject-wise shuffling
keys = list(P_processed2.keys())
random.seed(10)
random.shuffle(keys)

SuffledP_processed2 = dict()
for key in keys:
    SuffledP_processed2.update({key: P_processed2[key]})

In [ ]:
# Splitting data and labels in train/test
train_values2 = list(SuffledP_processed2.values())[0:31]
x_train2 = tf.concat(train_values2, axis = 0)
X_train2 = x_train2[:,:,:-1].numpy()
y_train2 = x_train2[:,0,-1].numpy()

test_values2 = list(SuffledP_processed2.values())[31:39]
x_test2 = tf.concat(test_values2, axis = 0)
X_test2 = x_test2[:,:,:-1].numpy()
y_test2 = x_test2[:,0,-1].numpy()

print("X_train:",X_train2.shape,
      "X_test:",X_test2.shape,
      "y_train:",y_train2.shape,
      "y_test:",y_test2.shape)

X_train: (2638, 576, 15) X_test: (681, 576, 15) y_train: (2638,) y_test: (681,)


In [ ]:
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_test2.pickle',X_test2)
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_train2.pickle',X_train2)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_test2.pickle',y_test2)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_train2.pickle',y_train2)

<_io.BufferedWriter name='/content/drive/MyDrive/Sam/File_saved/y_train2.pickle'>

# Method 3 - Resampling and BSC

In [ ]:
data = open_pickle('/content/drive/MyDrive/Sam/File_saved/P_dct.pickle')
markers = open_pickle('/content/drive/MyDrive/Sam/File_saved/markers_dct.pickle')

In [ ]:
# Selecting the sensor configuration
data_biomech_config = {}
for i in data:
  data_biomech_config[f'{i}_bmech'] = data[i].loc[:,('Frame','Right Hand x','Right Hand y','Right Hand z','Left Hand x','Left Hand y','Left Hand z',
                                                     'Right Foot x','Right Foot y','Right Foot z','Left Foot x','Left Foot y','Left Foot z',
                                                     'Pelvis x','Pelvis y','Pelvis z','Label')]

In [ ]:
# Empty dictionary for place holding
data_resample = {'P01_resample_bmech':{},'P02_resample_bmech':{},'P03_resample_bmech':{},'P04_resample_bmech':{},'P05_resample_bmech':{},'P07_resample_bmech':{},'P08_resample_bmech':{},
                 'P09_resample_bmech':{},'P10_resample_bmech':{},'P11_resample_bmech':{},'P12_resample_bmech':{},'P13_resample_bmech':{},'P14_resample_bmech':{},'P15_resample_bmech':{},
                 'P16_resample_bmech':{},'P17_resample_bmech':{},'P18_resample_bmech':{},'P19_resample_bmech':{},'P20_resample_bmech':{},'P21_resample_bmech':{},'P22_resample_bmech':{},
                 'P23_resample_bmech':{},'P24_resample_bmech':{},'P25_resample_bmech':{},'P26_resample_bmech':{},'P27_resample_bmech':{},'P28_resample_bmech':{},'P29_resample_bmech':{},
                 'P32_resample_bmech':{},'P33_resample_bmech':{},'P34_resample_bmech':{},'P35_resample_bmech':{},'P36_resample_bmech':{},'P37_resample_bmech':{},'P39_resample_bmech':{},
                 'P40_resample_bmech':{},'P41_resample_bmech':{},'P42_resample_bmech':{},'P43_resample_bmech':{}}

In [ ]:
# Runs some preprocessing steps and storing it into a new dictionary 'data_biomech'
data_biomech = {}
for j1,j2 in zip(markers, data_biomech_config):

  P_data = regroup_other(markers[j1], data_biomech_config[j2]) # regroups the 'SH', 'Pre', 'post' trials to the label 'Other'
  P_other = P_data[(P_data['Label'] == 'Other')] # creates a variable containing all the data labeled 'Other'
  P_other_all = other_class(P_other,400) # from the variable above, creates 15 trials of length 400
  P_data = P_data.drop(P_data[(P_data['Label'] == 'Do not use')].index) # removing all the data labeled as 'Do not use' (junk)
  P_data = P_data.drop(P_data[(P_data['Label'] == 'Other')].index) # removing all the data labeled as 'Other' from the original dataset
  P_data = pd.concat([P_data, P_other_all], ignore_index = True) # adding the 15 'Other' trials of length 400 to the original dataset
  data_biomech[f'{j2}'], data_classes3 = stand_labelencoder(P_data)

In [ ]:
data_classes3

array(['BH', 'OT', 'Other', 'Pass', 'Rest', 'SS', 'WS'], dtype=object)

In [ ]:
# This runs 3 different functions from the same 3 dictionaries
data_other = {} 
data_rest = {}
for m1,m2,m3 in zip(markers, data_resample, data_biomech):
  # Resampling the shooting trials to 400 frames and store them into the 'data_resample' dictionary
  resampling_data(m1,m2,m3, 400)
  # Creates a tensor of 'Other' from the 'data_biomech' dictionary and swap the axes to the required shape
  other = data_biomech[m3][-6000:,:]
  data_other[f'{m3}_other'] = create_tensor(other, 400)
  # Creates a tensor of 'Rest' from the data_biomech' dictionary and swap the axes to the required shape
  rest = data_biomech[m3][(data_biomech[m3][:,-1] == 4)]
  rest = rest[0:6000,:]
  data_rest[f'{m3}_rest'] = create_tensor(rest,400)

In [ ]:
# Converting the shots data from the 'data_resample' dictionary into tensor and store it into 'data_tensor' dictionary
data_tensor = {}
for i in data_resample:
  keys = data_resample[i].keys()
  t = np.array([data_resample[i][j] for j in keys])
  data_tensor[f'{i}_tensor'] = tf.convert_to_tensor(t, dtype = tf.float32)

In [ ]:
# Regrouping the shots, 'Other' and 'Rest' trials together in the 'data_tensor' dictionary
for n1,n2,n3 in zip(data_tensor, data_other, data_rest):
  data_tensor[n1] = tf.concat([data_tensor[n1],data_other[n2]], 0)
  data_tensor[n1] = tf.concat([data_tensor[n1],data_rest[n3]], 0)

In [ ]:
# Subject-wise shuffling
keys = list(data_tensor.keys())
random.seed(10)
random.shuffle(keys)

Shuffled_data_tensor = dict()
for key in keys:
    Shuffled_data_tensor.update({key: data_tensor[key]})

In [ ]:
# Splitting data and labels in train/test
train_values3 = list(Shuffled_data_tensor.values())[0:31]
x_train3 = tf.concat(train_values3, axis = 0)
X_train3 = x_train3[:,:,:-1].numpy()
y_train3 = x_train3[:,0,-1].numpy()

test_values3 = list(Shuffled_data_tensor.values())[31:39]
x_test3 = tf.concat(test_values3, axis = 0)
X_test3 = x_test3[:,:,:-1].numpy()
y_test3 = x_test3[:,0,-1].numpy()

print("X_train:",X_train3.shape,
      "X_test:",X_test3.shape,
      "y_train:",y_train3.shape,
      "y_test:",y_test3.shape)

X_train: (2638, 400, 15) X_test: (681, 400, 15) y_train: (2638,) y_test: (681,)


In [ ]:
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_test3.pickle',X_test3)
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_train3.pickle',X_train3)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_test3.pickle',y_test3)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_train3.pickle',y_train3)

<_io.BufferedWriter name='/content/drive/MyDrive/Sam/File_saved/y_train3.pickle'>

# Method 4 - Reframe to 576 and hands/feet sensor configuration (HFSC)

In [ ]:
data = open_pickle('/content/drive/MyDrive/Sam/File_saved/P_dct.pickle')
markers = open_pickle('/content/drive/MyDrive/Sam/File_saved/markers_dct.pickle')

In [ ]:
# Selecting the sensor configuration
data_HFSC = {}
for i in data:
  data_HFSC[f'{i}_HF'] = data[i].loc[:,('Frame','Right Hand x','Right Hand y','Right Hand z','Left Hand x','Left Hand y','Left Hand z',
                                        'Right Foot x','Right Foot y','Right Foot z','Left Foot x','Left Foot y','Left Foot z','Label')]

In [ ]:
# Preprocess function for method 4 and storing into 'P_processed4' dictionary
P_processed4 = {}
for l,n in zip(markers, data_HFSC):
  P_processed4[f'{n}_processed4'], data_classes4 = Preprocess(markers[l],data_HFSC[n])

In [ ]:
data_classes4

array(['BH', 'OT', 'Other', 'Pass', 'Rest', 'SS', 'WS'], dtype=object)

In [ ]:
# Subject-wise shuffling
keys = list(P_processed4.keys())
random.seed(10)
random.shuffle(keys)

SuffledP_processed4 = dict()
for key in keys:
    SuffledP_processed4.update({key: P_processed4[key]})

In [ ]:
# Splitting data and labels in train/test
train_values4 = list(SuffledP_processed4.values())[0:31]
x_train4 = tf.concat(train_values4, axis = 0)
X_train4 = x_train4[:,:,:-1].numpy()
y_train4 = x_train4[:,0,-1].numpy()

test_values4 = list(SuffledP_processed4.values())[31:39]
x_test4 = tf.concat(test_values4, axis = 0)
X_test4 = x_test4[:,:,:-1].numpy()
y_test4 = x_test4[:,0,-1].numpy()

print("X_train:",X_train4.shape,
      "X_test:",X_test4.shape,
      "y_train:",y_train4.shape,
      "y_test:",y_test4.shape)

X_train: (2638, 576, 12) X_test: (681, 576, 12) y_train: (2638,) y_test: (681,)


In [ ]:
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_test4.pickle',X_test4)
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_train4.pickle',X_train4)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_test4.pickle',y_test4)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_train4.pickle',y_train4)

<_io.BufferedWriter name='/content/drive/MyDrive/Sam/File_saved/y_train4.pickle'>

# Method 5 - Reframe to 576 and hands sensor configuration (HSC)

In [ ]:
data = open_pickle('/content/drive/MyDrive/Sam/File_saved/P_dct.pickle')
markers = open_pickle('/content/drive/MyDrive/Sam/File_saved/markers_dct.pickle')

In [ ]:
# Selecting the sensor configuration
data_HSC = {}
for i in data:
  data_HSC[f'{i}_H'] = data[i].loc[:,('Frame','Right Hand x','Right Hand y','Right Hand z','Left Hand x','Left Hand y','Left Hand z','Label')]

In [ ]:
# Preprocess function for method 5 and storing into 'P_processed5' dictionary
P_processed5 = {}
for l,n in zip(markers, data_HSC):
  P_processed5[f'{n}_processed5'], data_classes5 = Preprocess(markers[l],data_HSC[n])

In [ ]:
data_classes5

array(['BH', 'OT', 'Other', 'Pass', 'Rest', 'SS', 'WS'], dtype=object)

In [ ]:
# Subject-wise shuffling
keys = list(P_processed5.keys())
random.seed(10)
random.shuffle(keys)

SuffledP_processed5 = dict()
for key in keys:
    SuffledP_processed5.update({key: P_processed5[key]})

In [ ]:
# Splitting data and labels in train/test
train_values5 = list(SuffledP_processed5.values())[0:31]
x_train5 = tf.concat(train_values5, axis = 0)
X_train5 = x_train5[:,:,:-1].numpy()
y_train5 = x_train5[:,0,-1].numpy()

test_values5 = list(SuffledP_processed5.values())[31:39]
x_test5 = tf.concat(test_values5, axis = 0)
X_test5 = x_test5[:,:,:-1].numpy()
y_test5 = x_test5[:,0,-1].numpy()

print("X_train:",X_train5.shape,
      "X_test:",X_test5.shape,
      "y_train:",y_train5.shape,
      "y_test:",y_test5.shape)

X_train: (2638, 576, 6) X_test: (681, 576, 6) y_train: (2638,) y_test: (681,)


In [ ]:
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_test5.pickle',X_test5)
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_train5.pickle',X_train5)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_test5.pickle',y_test5)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_train5.pickle',y_train5)

<_io.BufferedWriter name='/content/drive/MyDrive/Sam/File_saved/y_train5.pickle'>

# Method 6 - Reframe to 576 and hands/feet/torso sensor configuration (HFTSC)

In [11]:
data = open_pickle('/content/drive/MyDrive/Sam/File_saved/P_dct.pickle')
markers = open_pickle('/content/drive/MyDrive/Sam/File_saved/markers_dct.pickle')

In [14]:
# Selecting the sensor configuration
data_HFTSC = {}
for i in data:
  data_HFTSC[f'{i}_HFT'] = data[i].loc[:,('Frame','T8 x','T8 y','T8 z','Right Hand x','Right Hand y','Right Hand z','Left Hand x','Left Hand y','Left Hand z',
                                         'Right Foot x','Right Foot y','Right Foot z','Left Foot x','Left Foot y','Left Foot z','Label')]

In [15]:
# Preprocess function for method 4 and storing into 'P_processed4' dictionary
P_processed6 = {}
for l,n in zip(markers, data_HFTSC):
  P_processed6[f'{n}_processed6'], data_classes6 = Preprocess(markers[l],data_HFTSC[n])

In [16]:
data_classes6

array(['BH', 'OT', 'Other', 'Pass', 'Rest', 'SS', 'WS'], dtype=object)

In [17]:
# Subject-wise shuffling
keys = list(P_processed6.keys())
random.seed(10)
random.shuffle(keys)

SuffledP_processed6 = dict()
for key in keys:
    SuffledP_processed6.update({key: P_processed6[key]})

In [18]:
# Splitting data and labels in train/test
train_values6 = list(SuffledP_processed6.values())[0:31]
x_train6 = tf.concat(train_values6, axis = 0)
X_train6 = x_train6[:,:,:-1].numpy()
y_train6 = x_train6[:,0,-1].numpy()

test_values6 = list(SuffledP_processed6.values())[31:39]
x_test6 = tf.concat(test_values6, axis = 0)
X_test6 = x_test6[:,:,:-1].numpy()
y_test6 = x_test6[:,0,-1].numpy()

print("X_train:",X_train6.shape,
      "X_test:",X_test6.shape,
      "y_train:",y_train6.shape,
      "y_test:",y_test6.shape)

X_train: (2638, 576, 15) X_test: (681, 576, 15) y_train: (2638,) y_test: (681,)


In [19]:
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_test6.pickle',X_test6)
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_train6.pickle',X_train6)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_test6.pickle',y_test6)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_train6.pickle',y_train6)

<_io.BufferedWriter name='/content/drive/MyDrive/Sam/File_saved/y_train6.pickle'>

# Method 7 - Reframe to 576 and hands/Torso sensor configuration (HTSC)

In [ ]:
data = open_pickle('/content/drive/MyDrive/Sam/File_saved/P_dct.pickle')
markers = open_pickle('/content/drive/MyDrive/Sam/File_saved/markers_dct.pickle')

In [20]:
# Selecting the sensor configuration
data_HTSC = {}
for i in data:
  data_HTSC[f'{i}_HT'] = data[i].loc[:,('Frame','T8 x','T8 y','T8 z','Right Hand x','Right Hand y','Right Hand z','Left Hand x','Left Hand y','Left Hand z',
                                        'Label')]

In [21]:
# Preprocess function for method 4 and storing into 'P_processed4' dictionary
P_processed7 = {}
for l,n in zip(markers, data_HTSC):
  P_processed7[f'{n}_processed7'], data_classes7 = Preprocess(markers[l],data_HTSC[n])

In [22]:
data_classes7

array(['BH', 'OT', 'Other', 'Pass', 'Rest', 'SS', 'WS'], dtype=object)

In [23]:
# Subject-wise shuffling
keys = list(P_processed7.keys())
random.seed(10)
random.shuffle(keys)

SuffledP_processed7 = dict()
for key in keys:
    SuffledP_processed7.update({key: P_processed7[key]})

In [24]:
# Splitting data and labels in train/test
train_values7 = list(SuffledP_processed7.values())[0:31]
x_train7 = tf.concat(train_values7, axis = 0)
X_train7 = x_train7[:,:,:-1].numpy()
y_train7 = x_train7[:,0,-1].numpy()

test_values7 = list(SuffledP_processed7.values())[31:39]
x_test7 = tf.concat(test_values7, axis = 0)
X_test7 = x_test7[:,:,:-1].numpy()
y_test7 = x_test7[:,0,-1].numpy()

print("X_train:",X_train7.shape,
      "X_test:",X_test7.shape,
      "y_train:",y_train7.shape,
      "y_test:",y_test7.shape)

X_train: (2638, 576, 9) X_test: (681, 576, 9) y_train: (2638,) y_test: (681,)


In [25]:
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_test7.pickle',X_test7)
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_train7.pickle',X_train7)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_test7.pickle',y_test7)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_train7.pickle',y_train7)

<_io.BufferedWriter name='/content/drive/MyDrive/Sam/File_saved/y_train7.pickle'>